# **Use YOLO-cls !**

## 0.미션


### (1) 미션1
여러분은 노트북에서 얼굴 인식 파일을 실행시키기 위해 **문제에 적합한** UltraLytics YOLO-cls 모델을 만들어야 합니다.

그 전에 가지고 있는 데이터셋을 **학습에 적합한 형태**로 바꿔야 합니다.

- 1) 데이터셋을 불러옵니다.
    - 데이터셋은 2가지입니다. 본인의 얼굴 이미지 파일, 다른 사람의 얼굴 이미지 파일.
- 2) 데이터셋을 전처리합니다.
    - UltraLytics YOLO-cls 모델에서 요구하는 데이터셋 폴더의 구조가 있습니다.
    - [UltraLytics YOLO-cls 모델의 데이터셋 구조 링크](https://docs.ultralytics.com/datasets/classify/)

### (2) 미션2
데이터셋의 폴더 구조를 **학습에 적합한 형태**로 만들었다면, **사전 학습된 UltraLytics YOLO-cls 모델**에 Transfer Learning을 수행합니다.

- 1) UltraLytics YOLO-cls 모델 선택
    - 세부 모델로 n, s, m, l, x가 있습니다. n가 가장 빠르고, x가 가장 연산량이 많습니다.
    - [UltraLytics YOLO-cls 모델 링크](https://docs.ultralytics.com/tasks/classify/)
- 2) 선택한 UltraLytics YOLO-cls 모델로 학습을 진행합니다.
    - [UltraLytics YOLO 학습 명령어 링크](https://docs.ultralytics.com/modes/train/#train-settings)
- 3) 학습이 완료되면 추론을 진행합니다.
    - [UltraLytics YOLO 추론 명령어 링크](https://docs.ultralytics.com/modes/predict/#inference-arguments)
- 4) 해당 UltraLytics YOLO-cls 모델을 **반드시** 저장합니다.
    - 모델을 **반드시** 저장하세요.
    - .pt 형태로 Colab에 저장이 될 것입니다. 해당 파일을 **로컬에 다운로드** 하세요.

## 1.환경설정

* 세부 요구사항
    - 경로 설정 : 구글콜랩
        * 구글 드라이브 바로 밑에 project4 폴더를 만드세요.
        * 데이터 파일을 복사해 넣습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.

### (1) 경로 설정

* 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/project4'

### (2) 라이브러리 설치 및 불러오기

* 라이브러리 로딩

In [3]:
## colab에서 세션 재시작을 요구하는 팝업이 뜨면 재시작 누르세요.
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 36.5 MB/s eta 0:00:00


## 2.미션1

여러분은 노트북에서 얼굴 인식 파일을 실행시키기 위해 **문제에 적합한** UltraLytics YOLO-cls 모델을 만들어야 합니다.

그 전에 가지고 있는 데이터셋을 **학습에 적합한 형태**로 바꿔야 합니다.

- 1) 데이터셋을 불러옵니다.
    - 데이터셋은 2가지입니다. 본인의 얼굴 이미지 파일, 다른 사람의 얼굴 이미지 파일.
- 2) 데이터셋을 전처리합니다.
    - UltraLytics YOLO-cls 모델에서 요구하는 데이터셋 폴더의 구조가 있습니다.
    - [UltraLytics YOLO-cls 모델의 데이터셋 구조 링크](https://docs.ultralytics.com/datasets/classify/)

### (1) 데이터셋 불러오기

* **세부 요구사항**
    - 데이터셋을 불러옵니다.
        - 데이터셋은 두 개의 압축 파일이어야 합니다.
            1. lfw-deepfunneled.zip : Labeled Faces in the Wild 데이터셋
                - 압축 파일을 로컬에 다운로드 받아서 **어떤 구조**인지 확인하세요.
            2. 여러분의 얼굴 이미지 데이터셋
                - 여러분의 얼굴 이미지가 담긴 **압축 파일**을 **Google Drive에 업로드** 하기를 권장합니다.
                    - 이미지 파일 하나하나 업로드 하면 시간이 오래 걸립니다.
    - 데이터셋 압축 파일을 **Colab에 폴더를 생성한 후 해제**하세요.
        - 데이터셋 폴더를 **본인 얼굴 폴더, LFW 폴더로 나누어** 생성하는 것을 권장합니다.
        - 만일 두 압축 파일을 하나의 폴더에 모두 해제하면 전처리가 더 까다로워질 것입니다.
    - 예시 코드에서 사용한 라이브러리
        - os, zipfile

In [4]:
import os, zipfile
from tqdm import tqdm
import shutil

In [5]:
def unzip(data_fp, my_fp):
    data_types = ['lfw-deepfunneled', f'{my_fp}']
    for data_type in data_types:
        if not os.path.exists(data_fp + f'/{data_type}.zip'):
            raise FileNotFoundError(f'{data_type}.zip file required')

        # save directory
        data_type_head = data_type.split('-')[0]
        save_dir = f'{data_fp}/{data_type_head}-raw'
        os.makedirs(save_dir, exist_ok=True)
        # Unzip
        with zipfile.ZipFile(f'{data_fp}/{data_type}.zip') as myzip:
            file_list = myzip.namelist()
            print(f'Unzip {data_type}.zip started...')
            for f in tqdm(file_list):
                # directory가 아니고, 확장자가 .jpg인 파일들만 호출
                if not f.endswith('/') and f.lower().endswith('.jpg'):
                    # 폴더를 제외한 순수 파일명만을 호출
                    file_name = os.path.basename(f)
                    # 이미지 파일만 unzip
                    if not file_name.startswith('._'):
                        fp = os.path.join(save_dir, file_name)
                        with myzip.open(f) as source, open(fp, 'wb') as target:
                            target.write(source.read())
        print('Done!')

In [6]:
os.makedirs('/content/Datasets', exist_ok=True)
shutil.copy(src='/content/drive/MyDrive/mini4/Datasets/Keras/lfw-deepfunneled.zip',
            dst='/content/Datasets/lfw-deepfunneled.zip')
shutil.copy(src='/content/drive/MyDrive/mini4/Datasets/Keras/my-face.zip',
            dst='/content/Datasets/my-face.zip')

'/content/Datasets/my-face.zip'

#### 1) 본인 얼굴 이미지 데이터셋 불러오기

#### 2) 다른 얼굴 이미지 데이터셋 불러오기

In [7]:
unzip('/content/Datasets', 'my-face')

Unzip lfw-deepfunneled.zip started...


100%|██████████| 43719/43719 [00:01<00:00, 23070.09it/s]


Done!
Unzip my-face.zip started...


100%|██████████| 2487/2487 [00:00<00:00, 3555.15it/s]

Done!


In [8]:
os.remove('/content/Datasets/lfw-deepfunneled.zip')
os.remove('/content/Datasets/my-face.zip')

### (2) 데이터셋 전처리

* **세부 요구사항**
    - 데이터셋을 전처리 합니다.
        - YOLO-cls 모델이 요구하는 폴더 구조를 만듭니다.
            1. Datasets라는 폴더를 생성합니다.
            2. Training set, Validation set, Test set(선택 사항) 각 데이터셋이 들어갈 폴더를 생성합니다.
            3. 각 데이터셋 폴더에 분류할 클래스의 이름을 가진 폴더를 생성합니다.
        - 폴더 구조에 맞게 데이터를 분배합니다.
    - 예시 코드에서 사용한 라이브러리
        - os, glob, random, shutil, numpy

In [9]:
import random, glob

In [10]:
def split_imgs(data_fp, val_size, test_size):
    ## Create new folder
    os.makedirs(f'{data_fp}/train', exist_ok=True)
    os.makedirs(f'{data_fp}/test', exist_ok=True)

    ## Load images
    img_lfw = sorted(glob.glob(f'{data_fp}/lfw-raw/*'))
    img_my = sorted(glob.glob(f'{data_fp}/my-raw/*'))
    print('The number of images')
    print('LFW: ', len(img_lfw))
    print('My: ', len(img_my))

    ## Shuffle images before train/test split
    random.seed(2024)
    random.shuffle(img_lfw)
    random.shuffle(img_my)

    ## Split data
    for img_fp_list in [img_lfw, img_my]:
        total_len = len(img_fp_list)
        split_idx1 = int(total_len * val_size)
        split_idx2 = int(total_len * (val_size + test_size))
        list_val = img_fp_list[:split_idx1]
        list_te = img_fp_list[split_idx1:split_idx2]
        list_tr = img_fp_list[split_idx2:total_len]

        # Case 1. lfw-raw
        if img_fp_list == img_lfw:
            # Create directories
            os.makedirs(f'{data_fp}/train/other', exist_ok=True)
            os.makedirs(f'{data_fp}/test/other', exist_ok=True)
            os.makedirs(f'{data_fp}/val/other', exist_ok=True)
            # Case 1-1. train/other
            print('Copy train/other data...')
            for fp in tqdm(list_tr):
                f_name = os.path.basename(fp)
                shutil.copy(src=fp, dst=f'{data_fp}/train/other/'+f_name)
            print('Done!')

            # Case 1-2. test/other
            print('Copy test/other data...')
            for fp in tqdm(list_te):
                f_name = os.path.basename(fp)
                shutil.copy(src=fp, dst=f'{data_fp}/test/other/'+f_name)
            print('Done!')

            # Case 1-3. val/other
            print('Copy val/other data...')
            for fp in tqdm(list_val):
                f_name = os.path.basename(fp)
                shutil.copy(src=fp, dst=f'{data_fp}/val/other/'+f_name)
            print('Done!')

        # Case 2. my-raw
        if img_fp_list == img_my:
            # Create directories
            os.makedirs(f'{data_fp}/train/my', exist_ok=True)
            os.makedirs(f'{data_fp}/test/my', exist_ok=True)
            os.makedirs(f'{data_fp}/val/my', exist_ok=True)
            # Case 2-1. train/my
            print('Copy train/my data...')
            for fp in tqdm(list_tr):
                f_name = os.path.basename(fp)
                shutil.copy(src=fp, dst=f'{data_fp}/train/my/'+f_name)
            print('Done!')

            # Case 2-2. test/my
            print('Copy train/my data...')
            for fp in tqdm(list_te):
                f_name = os.path.basename(fp)
                shutil.copy(src=fp, dst=f'{data_fp}/test/my/'+f_name)
            print('Done!')

            # Case 2-3. val/my
            print('Copy val/my data...')
            for fp in tqdm(list_val):
                f_name = os.path.basename(fp)
                shutil.copy(src=fp, dst=f'{data_fp}/val/my/'+f_name)
            print('Done!')

#### 1) 모델이 요하는 구조의 폴더 생성

#### 2) 각 폴더에 이미지 데이터 옮기기

In [11]:
split_imgs('/content/Datasets', val_size=0.16, test_size=0.2)

The number of images
LFW:  13233
My:  2487
Copy train/other data...


100%|██████████| 8470/8470 [00:00<00:00, 9287.23it/s]


Done!
Copy test/other data...


100%|██████████| 2646/2646 [00:00<00:00, 8383.38it/s]


Done!
Copy val/other data...


...

Done!
Copy train/my data...


100%|██████████| 498/498 [00:00<00:00, 7349.03it/s]


Done!
Copy val/my data...


100%|██████████| 397/397 [00:00<00:00, 6475.06it/s]

Done!


## 3.미션2

데이터셋의 폴더 구조를 **학습에 적합한 형태**로 만들었다면, **사전 학습된 UltraLytics YOLO-cls 모델**에 Transfer Learning을 수행합니다.

- 1) UltraLytics YOLO-cls 모델 선택
    - 세부 모델로 n, s, m, l, x가 있습니다. n가 가장 빠르고, x가 가장 연산량이 많습니다.
    - [UltraLytics YOLO-cls 모델 링크](https://docs.ultralytics.com/tasks/classify/)
- 2) 선택한 UltraLytics YOLO-cls 모델로 학습을 진행합니다.
    - [UltraLytics YOLO 학습 명령어 링크](https://docs.ultralytics.com/modes/train/#train-settings)
- 3) 학습이 완료되면 추론을 진행합니다.
    - [UltraLytics YOLO 추론 명령어 링크](https://docs.ultralytics.com/modes/predict/#inference-arguments)
- 4) 해당 UltraLytics YOLO-cls 모델을 **반드시** 저장합니다.
    - 모델을 **반드시** 저장하세요.
    - .pt 형태로 Colab에 저장이 될 것입니다. 해당 파일을 **로컬에 다운로드** 하세요.

#### (1) UltraLytics YOLO-cls 모델 선택

* **세부 요구사항**
    - 세부 모델로 n, s, m, l, x가 있습니다. n가 가장 빠르고, x가 가장 연산량이 많습니다.
    - [UltraLytics YOLO-cls 모델 링크](https://docs.ultralytics.com/tasks/classify/)

In [12]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [14]:
# Load a model
# model = YOLO("yolo11n-cls.pt")
model = YOLO('yolo11l-cls.pt')

100%|██████████| 27.2M/27.2M [00:00<00:00, 203MB/s]


#### (2) UltraLytics YOLO-cls 모델 학습

* **세부 요구사항**
    - 선택한 UltraLytics YOLO-cls 모델로 학습을 진행합니다.
    - [UltraLytics YOLO 학습 명령어 링크](https://docs.ultralytics.com/modes/train/#train-settings)

In [ ]:
results = model.train(data='/content/Datasets', epochs=20, imgsz=160, patience=5, pretrained=True)

Ultralytics 8.3.24 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolo11l-cls.pt, data=/content/Datasets, epochs=20, time=None, patience=5, batch=16, imgsz=160, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

100%|██████████| 5.35M/5.35M [00:00<00:00, 366MB/s]


AMP: checks passed ✅


train: Scanning /content/Datasets/train... 10062 images, 0 corrupt: 100%|██████████| 10062/10062 [00:04<00:00, 2480.79it/s]

train: New cache created: /content/Datasets/train.cache


...

      15/20     0.782G   0.001128         14        160: 100%|██████████| 629/629 [00:42<00:00, 14.63it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:02<00:00, 37.70it/s]

                   all          1          1



      Epoch    GPU_mem       loss  Instances       Size


      16/20     0.795G  0.0004908         14        160: 100%|██████████| 629/629 [00:43<00:00, 14.61it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:02<00:00, 36.53it/s]

                   all          1          1


#### (3) UltraLytics YOLO-cls 추론

* **세부 요구사항**
    - 학습이 완료되면 추론을 진행합니다.
    - [UltraLytics YOLO 추론 명령어 링크](https://docs.ultralytics.com/modes/predict/)

In [ ]:
img_fp_list = glob.glob('/content/Datasets/test/*/*.jpg')
print(len(img_fp_list), img_fp_list[:5])

In [ ]:
pred = model.predict(img_fp_list[:5])

In [ ]:
pred = model(img_fp_list)

In [ ]:
import numpy as np
import torch

In [ ]:
print(pred[-5])

In [ ]:
pred_ = np.array([p_.probs.data.detach().cpu().numpy() for p_ in pred])

In [ ]:
print(pred_[:5])

In [ ]:
y_real = [0 for fp in img_fp_list if fp.split('/')[-2] == 'my'] + [1 for fp in img_fp_list if fp.split('/')[-2] == 'other']
y_pred = np.argmax(pred_, axis=1)

In [ ]:
print(y_pred[:5])
print(y_pred[-5:])
print(y_real[:5])
print(y_real[-5:])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_real, y_pred))
print(confusion_matrix(y_real, y_pred))

#### (4) UltraLytics YOLO-cls 모델 저장

* **세부 요구사항**
    - 모델을 **반드시** 저장하세요.
    - .pt 형태로 Colab에 저장이 될 것입니다. 해당 파일을 **로컬에 다운로드** 하세요.

In [ ]:
os.makedirs('/content/wts', exist_ok=True)
# torch.save(model.state_dict(), '/content/wts/model_YOLO-cls.pt')
model.save('/content/wts/model_YOLO-cls_L.pt')

In [ ]:
import torch

In [28]:
shutil.copy(src='/content/wts/model_YOLO-cls.pt',
            dst='/content/drive/MyDrive/mini4/weights/model_YOLO-cls_L_best.pt')

'/content/drive/MyDrive/mini4/weights/model_YOLO-cls_best2.pt'